In [ ]:
# import modules and define functions
from pyscf import gto, scf
import pandas as pd
import numpy as np
from scipy import linalg
import os
from functools import reduce
os.environ['OMP_NUM_THREADS'] = "1"

def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

# Following step 1, 2, and 3

In [ ]:
# Step 1
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74' #{R_A}, {Z_A}
basis = 'sto-3g' # {\phi_\mu}
h2_0_1 = gto.Mole()
h2_0_1.build(atom=atom,basis=basis,charge=0,spin=0)

In [ ]:
# Step 2
# obtain the matrices that are defined as soon as the molecule is defined
s1e = h2_0_1.intor_symmetric('int1e_ovlp') # S_\mu\nv
t = h2_0_1.intor_symmetric('int1e_kin')
en = h2_0_1.intor_symmetric('int1e_nuc')
h1e = t + en # H^core_\mu\nv
eri = h2_0_1.intor_symmetric('int2e') # (\mu\nu|\lambda\sigma)

In [ ]:
# Step 3
# obtain transformation matrices
evals, evecs = linalg.eig(s1e) # diagonalize S_\mu\nv
U = np.array([evec/linalg.norm(evec) for evec in evecs.T]).T # obtain U
X = np.array([U[:,i]/(evals[i] ** 0.5) for i in range(U.shape[1])]).T # X = U*s^-1/2

# Following step 3 to 6

In [ ]:
# Step 4
# obtain initial guess density matrix
c1 = 0
c2 = (1+(c1**2)*(s1e[0][1]**2-1)) ** 0.5 - c1*s1e[0][1]
mo_coeff = np.array([[c1, c1], [c2, -c2]])
mo_occ = np.array([2, 0])
dms = make_rdm1(mo_coeff, mo_occ) # calculate P

In [ ]:
# Step 5
# initiate mean field calculations
h2_0_1_rhf = scf.RHF(h2_0_1)
# obtain the needed matrices
vhf = h2_0_1_rhf.get_veff(h2_0_1, dms)
H = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, h1e, mo_coeff]) # calculate H^core
V = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, vhf, mo_coeff]) # calculate G
# Fock matrix is essentially the sum of 
# the kinetic and potential energy of electron.
# The off-diagonals reaching zero means that 
# the electrons arrive at the best configuration
F = H+V # calculate F

# Following step 7 to 10

In [ ]:
# Step 7
# transform F by X
F_prime = reduce(lambda a,b: np.dot(a,b), [X.T.conjugate(), F, X])


In [ ]:
# Step 8
# diagonalize F_prime to obtain C' and e
e, C_prime = linalg.eig(F_prime) 

In [ ]:
# Step 9
C = np.dot(X, C_prime) # calculate 

In [ ]:
# Step 10
# obtain the new P
mo_occ = np.array([2, 0])
dms_new = make_rdm1(C, mo_occ)

# Step 12

In [ ]:
vhf_new = h2_0_1_rhf.get_veff(h2_0_1, dms_new)
H_new = reduce(lambda a,b: np.dot(a,b), [C.T, h1e, C]) # new H^core
V_new = reduce(lambda a,b: np.dot(a,b), [C.T, vhf_new, C]) # new G
F_new = H_new+V_new # new F
mo_e, mo_coeff = linalg.eig(F_new) # obtain orbital energies
H1e = np.sum(np.diagonal(H_new)[mo_occ>0])
J2e = np.sum(np.diagonal(V_new)[mo_occ>0])
E_ele = H1e*2+J2e # calculate electronic energy
e_nuc = h2_0_1.get_enuc()
E_tot = E_ele + e_nuc # calculate total energy
print('Orbital energies are %s' %('. '.join(['MO%s:%.4f Eh' %(i, e) for i, e in enumerate(mo_e)])))
print('Total electronic energy is %.4f Eh' %(E_ele))
print('Total energy is %.4f Eh' %(E_tot))

# Compare to a coded SCF procedure

In [ ]:
h2_0_1_rhf.run() # perform a coded SCF procedure
mo_energy = h2_0_1_rhf.mo_energy # obtain MO energies
e_elec = h2_0_1_rhf.energy_elec()[0] # obtain electronic energies
e_tot = h2_0_1_rhf.e_tot # obtain total energy
print('Orbital energies are %s' %('. '.join(['MO%s:%.4f Eh' %(i, e) for i, e in enumerate(mo_energy)])))
print('Total electronic energy is %.4f Eh' %(e_elec))
print('Total energy is %.4f Eh' %(e_tot))